In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import gc
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split, KFold, StratifiedGroupKFold, StratifiedKFold
from sklearn.metrics import f1_score
from torch.nn.utils.rnn import pad_sequence
import cv2
import io
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from math import sin,cos,pi
from sklearn.metrics import roc_auc_score, auc, roc_curve
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from transformers import get_cosine_schedule_with_warmup
from catboost import CatBoostClassifier, Pool, cv
from copy import deepcopy
import wandb
import timm 

In [2]:
timm.list_pretrained()[::-1]

['xcit_tiny_24_p16_384.fb_dist_in1k',
 'xcit_tiny_24_p16_224.fb_in1k',
 'xcit_tiny_24_p16_224.fb_dist_in1k',
 'xcit_tiny_24_p8_384.fb_dist_in1k',
 'xcit_tiny_24_p8_224.fb_in1k',
 'xcit_tiny_24_p8_224.fb_dist_in1k',
 'xcit_tiny_12_p16_384.fb_dist_in1k',
 'xcit_tiny_12_p16_224.fb_in1k',
 'xcit_tiny_12_p16_224.fb_dist_in1k',
 'xcit_tiny_12_p8_384.fb_dist_in1k',
 'xcit_tiny_12_p8_224.fb_in1k',
 'xcit_tiny_12_p8_224.fb_dist_in1k',
 'xcit_small_24_p16_384.fb_dist_in1k',
 'xcit_small_24_p16_224.fb_in1k',
 'xcit_small_24_p16_224.fb_dist_in1k',
 'xcit_small_24_p8_384.fb_dist_in1k',
 'xcit_small_24_p8_224.fb_in1k',
 'xcit_small_24_p8_224.fb_dist_in1k',
 'xcit_small_12_p16_384.fb_dist_in1k',
 'xcit_small_12_p16_224.fb_in1k',
 'xcit_small_12_p16_224.fb_dist_in1k',
 'xcit_small_12_p8_384.fb_dist_in1k',
 'xcit_small_12_p8_224.fb_in1k',
 'xcit_small_12_p8_224.fb_dist_in1k',
 'xcit_nano_12_p16_384.fb_dist_in1k',
 'xcit_nano_12_p16_224.fb_in1k',
 'xcit_nano_12_p16_224.fb_dist_in1k',
 'xcit_nano_12_p8_3

In [2]:
class CFG:
    class data:
        train_data = 'data/train.csv'
        train_path ='./data/train'
        test_data = 'data/sample_submission.csv'
        test_path ='./data/test'
        num_workers = 8
        img_size = 384
        nfolds = 5
        batch_size = 32
        eval_batch_size = 128
        seed = 56
    class model:
        model = 'tf_efficientnetv2_l.in21k_ft_in1k'
        pretrained = True
        optim = torch.optim.AdamW
        global_pool = 'avg' # 'avg', 'max', 'avgmax', 'catavg'
        drop_path_rate = 0.3 
        cls_drop = 0.2
        num_chanels = 3
        num_labels = 10
        hidden_size = 1280# 2152 - effnetv2_m
        scheduler = 'cosine'
        max_epoches = 10
        lr = lr_head = 1e-4
        num_cycles = 0.5
        warmup_ratio = 0.0
        eps = 1e-12
        weight_decay = 0.0
        weight_decay_head = 0.0
        betas = (0.9, 0.999)
    seed = 56
    gen_fold_number = 0
    fold_number = 0
    
class Transforms:
    transforms_train = A.Compose([
            A.Resize(CFG.data.img_size,CFG.data.img_size),
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            #A.ShiftScaleRotate(
            #    rotate_limit=(-10, 10),
            #    scale_limit=(-0.1,0.1),
            #    shift_limit=(-0.05, 0.05),
            #    p=0.25
            #),
            #A.Downscale(p=0.25,scale_range=(0.25, 0.5)),
            #A.RandomBrightnessContrast(brightness_limit=(0.2, -0.2), contrast_limit=(-0.2, 0.2),p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
            ToTensorV2()
        ])
    transforms_val = A.Compose([
            A.Resize(CFG.data.img_size,CFG.data.img_size),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
            ToTensorV2()
        ])
    transforms_test = A.Compose([
            A.Resize(CFG.data.img_size,CFG.data.img_size),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
            ToTensorV2()
        ])

def set_wandb_cfg():
    config = {}
    for k,v in CFG.model.__dict__.items():
        if '__' not in k:
            config[k] = v
    for k,v in CFG.data.__dict__.items():
        if '__' not in k:
            config[k] = v
    config['fold_number'] = CFG.fold_number
    return config

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
def make_df(path,root=CFG.data.train_path,is_test=False):
    data = pd.read_csv(path)
    if not is_test:
        data['labels'] = data['class_id']
    else:
        data['labels'] = 0
    data['image_path'] = root + '/' + data['image_name']
    return data[['labels','image_path']]

In [4]:
class PLDataset(Dataset):
    def __init__(self, df, transforms):
        super().__init__()
        self.cfg = CFG.data
        self.data = df
        self.transforms = transforms
        
    def __getitem__(self, index):
        row = self.data.iloc[index]
        image = cv2.imread(row['image_path'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
            
        return {
            'image': image,
            'labels': row['labels']
        }
    
    def __len__(self):
        return len(self.data)

In [5]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.data
        self.is_setup = False
        self.is_prepared = False
            
    def prepare_data(self):
        if self.is_prepared: return None
        self.df = make_df(self.cfg.train_data)
        self.test_df = make_df(self.cfg.test_data,root=self.cfg.test_path,is_test=True)
        self.transforms = Transforms
        self.is_prepared = True
        
    def setup(self, stage: str):
        if self.is_setup: return None
        kf = StratifiedKFold(n_splits=self.cfg.nfolds, shuffle=True, random_state=self.cfg.seed)
        splits = [(x,y) for x,y in  kf.split(self.df,self.df['labels'])][CFG.fold_number]
        self.train_df, self.val_df = self.df.iloc[splits[0]], self.df.iloc[splits[1]]
        self.train_dataset = PLDataset(self.train_df,self.transforms.transforms_train)
        self.val_dataset = PLDataset(self.val_df,self.transforms.transforms_val)
        self.predict_dataset = PLDataset(self.test_df,self.transforms.transforms_test)
        self.is_setup = True
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                         batch_size=self.cfg.batch_size,
                         num_workers=self.cfg.num_workers,
                         pin_memory=True,
                         shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.cfg.eval_batch_size,
                          num_workers=self.cfg.num_workers,
                          pin_memory=True,
                          shuffle=False)
    
    def predict_dataloader(self):
        return DataLoader(self.predict_dataset,
                          batch_size=self.cfg.eval_batch_size,
                          num_workers=self.cfg.num_workers,
                          pin_memory=True,
                          shuffle=False)

In [6]:
class AverageMeter():
    def __init__(self):
        self.preds = []
        self.labels = []
        self.history = []
    
    def update(self,y_t,y_p):
        self.labels += y_t
        self.preds += y_p
        
    def clean(self):
        self.preds = []
        self.labels = []

    def calc_metrics(self):
        metrics = {}
        metrics['f1_macro'] = f1_score(self.labels, self.preds, average='macro')
        self.history.append(metrics)
        return metrics

In [7]:
class ImageEncoder(nn.Module):
    def __init__(self):
        super(ImageEncoder,self).__init__()
        self.cfg = CFG.model
        self.encoder = timm.create_model(
                self.cfg.model,
                pretrained=self.cfg.pretrained,
                in_chans=self.cfg.num_chanels,
                num_classes=-1,
                drop_path_rate=self.cfg.drop_path_rate,
                global_pool=self.cfg.global_pool
        )
        self.cls_drop = nn.Dropout(self.cfg.cls_drop)
        self.fc = nn.Linear(self.cfg.hidden_size, self.cfg.num_labels)
        self._init_weights(self.fc)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def forward(self, image, return_features=False):
        features = self.encoder(image)
        if return_features:
            return features
        logits = self.fc(self.cls_drop(features))
        return logits

In [8]:
class PLModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.model
        self.model = ImageEncoder()
        self.avg_meter = AverageMeter()
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, batch):
        output = self.model(batch['image'])
        return output

    def training_step(self, batch, i):
        logits = self(batch)
        loss = self.criterion(logits,batch['labels'])
        self.log('train_loss', loss.item())
        return loss
            
    def validation_step(self, batch, i):
        logits = self(batch)
        loss = self.criterion(logits,batch['labels'])
        self.log('val_loss',loss.item())
        preds = logits.argmax(dim=-1).tolist()
        self.avg_meter.update(batch['labels'].tolist(),preds)
    
    def predict_step(self, batch, i):
        logits = self(batch)
        return logits.softmax(dim=-1).tolist()
                
    def on_validation_epoch_end(self):
        metrics = self.avg_meter.calc_metrics()
        self.log_dict(metrics)
        self.avg_meter.clean()
            
    def configure_optimizers(self):        
        optimizer_parameters = [
            {'params': self.model.encoder.parameters(),
             'lr': self.cfg.lr, 'weight_decay': self.cfg.weight_decay},
            {'params': self.model.fc.parameters(),
             'lr': self.cfg.lr_head, 'weight_decay': self.cfg.weight_decay_head}
        ]
        
        optim = self.cfg.optim(
            optimizer_parameters,
            lr=self.cfg.lr,
            betas=self.cfg.betas,
            weight_decay=self.cfg.weight_decay,
            eps=self.cfg.eps
        )
        
        if self.cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warmup_ratio,
                                                        num_cycles=self.cfg.num_cycles)
        elif self.cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warmup_ratio)
        else:
            return optim
        
        scheduler = {'scheduler': scheduler,'interval': 'step', 'frequency': 1}

        return [optim], [scheduler]

In [9]:
pl.seed_everything(56)
CFG.fold_number = 0

Seed set to 56


In [10]:
dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

In [11]:
CFG.model.num_training_steps = len(dm.train_dataloader()) * CFG.model.max_epoches
model = PLModule().cuda()

In [12]:
wandb.login(key="31520b01739d418e5d77a11fd8a79a70b189b8bc")
os.environ['WANDB_API_KEY'] = "31520b01739d418e5d77a11fd8a79a70b189b8bc"
wandb.init(project='NTO_AI_2425_1',name='tf_efficientnetv2_l.in21k_ft_in1k',config=set_wandb_cfg())

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: andrewkhl (andlh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')
checkpoint_cb = pl.callbacks.ModelCheckpoint(
    dirpath=f'./outputs_{0}/',
    filename='model_{epoch:02d}-{prauc:.4f}',
    monitor='prauc',
    mode='max',
    save_last=True
)

trainer = pl.Trainer(
    accelerator="gpu",
    precision=32,
    callbacks = [lr_monitor],
    logger = pl.loggers.WandbLogger(save_code=True),
    enable_checkpointing=False,
    log_every_n_steps=1,
    min_epochs=1,
    devices=1,
    check_val_every_n_epoch=1,
    max_epochs=CFG.model.max_epoches
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(model,datamodule=dm)

You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | ImageEncoder     | 117 M  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
117 M     Trainable params
0         Non-trainable params
117 M     T

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [19]:
preds = trainer.predict(model,dm.predict_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [20]:
preds = np.concatenate(preds)

In [23]:
preds = preds.argmax(axis=-1)

In [25]:
sub = pd.read_csv('data/sample_submission.csv')

In [27]:
sub['predicted_class'] = preds

In [29]:
sub.to_csv('nto_ii_sub0.csv',index=False)